In [2]:
####################################Imports and TimeStamp###################################################
import feedparser as fp, pandas as pd, datetime, time, string, requests as r
from bs4 import BeautifulSoup 
from functools import reduce

#Write the Date&Time that the Rip Occured
download_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

file = open('dateDownloaded.txt','w')
file.write(download_time)
file.close()

###################################################################################################################

In [4]:
######################################Load in all of the Sentiment Files###########################################


#Load in the Huffington Post DataFrame
huffington_df = pd.read_csv('DataFramed_Source_Text/HuffingtonPostSentiment.csv')

###################################################################################################################

In [5]:
################################################Pull Stock Data#####################################################
company_info_df = pd.read_csv('DataFramed_Source_Text/Target_Company.csv')
ticker_symbol = company_info_df.iloc[0]['Stock Symbol']

raw_stock_quote = r.get('http://dev.markitondemand.com/Api/v2/Quote/json?symbol=' + ticker_symbol).json()
stock_quote = pd.DataFrame(raw_stock_quote,index=[0])
stock_quote.head()
###################################################################################################################

,Change,ChangePercent,ChangePercentYTD,ChangeYTD,High,LastPrice,Low,MSDate,MarketCap,Name,Open,Status,Symbol,Timestamp,Volume
0,0.1,0.092132,3.211096,105.26,109.77,108.64,108.17,42468.665972,602363497120,Apple Inc,108.91,SUCCESS,AAPL,Fri Apr 8 15:59:00 UTC-04:00 2016,1895413
